In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
DATA_DIR = os.getcwd()
FEATURES_DIR = DATA_DIR

# Load eGeMAPS
egemaps_features = np.load(os.path.join(FEATURES_DIR, r"opensmile\egemaps_features.npy"))
egemaps_labels = np.load(os.path.join(FEATURES_DIR, r"opensmile\egemaps_labels.npy"))

# Load Wav2Vec2
wav2vec2_features = np.load(os.path.join(FEATURES_DIR, r"wav2vec2 features\wav2vec2_features.npy"))
wav2vec2_labels = np.load(os.path.join(FEATURES_DIR, r"wav2vec2 features\labels.npy"))

# Load WavLM
wavlm_features = np.load(os.path.join(FEATURES_DIR, r"wavLm features\wavlm_features.npy"))
wavlm_labels = np.load(os.path.join(FEATURES_DIR, r"wavLm features\wavlm_labels.npy"))

print("✅ Features loaded successfully:")
print("eGeMAPS:", egemaps_features.shape)
print("Wav2Vec2:", wav2vec2_features.shape)
print("WavLM:", wavlm_features.shape)


✅ Features loaded successfully:
eGeMAPS: (6930, 88)
Wav2Vec2: (6930, 768)
WavLM: (6930, 768)


In [7]:
# ============================================
# Cell 3: Concatenate All Features (Final Fixed Version)
# ============================================

from sklearn.preprocessing import LabelEncoder
import numpy as np

# Convert numeric labels (0/1) in wav2vec2 to string form
# We'll detect and map automatically
if np.issubdtype(wav2vec2_labels.dtype, np.number):
    unique_labels = np.unique(wav2vec2_labels)
    if len(unique_labels) == 2:  # Binary case
        mapping = {0: "Control", 1: "Dementia"} if "Control" in egemaps_labels else {0: "Class0", 1: "Class1"}
        wav2vec2_labels = np.array([mapping[int(l)] for l in wav2vec2_labels])
    else:
        wav2vec2_labels = wav2vec2_labels.astype(str)

# Convert all to string for consistency
egemaps_labels = egemaps_labels.astype(str)
wavlm_labels = wavlm_labels.astype(str)

# Sanity check: All should have same set of class names
print("Unique labels per source:")
print("eGeMAPS:", np.unique(egemaps_labels))
print("Wav2Vec2:", np.unique(wav2vec2_labels))
print("WavLM:", np.unique(wavlm_labels))

# Encode with one label encoder
le = LabelEncoder()
le.fit(egemaps_labels)  # using eGeMAPS as base

egemaps_labels_encoded = le.transform(egemaps_labels)
wav2vec2_labels_encoded = le.transform(wav2vec2_labels)
wavlm_labels_encoded = le.transform(wavlm_labels)

# Double-check all match
assert np.array_equal(egemaps_labels_encoded, wav2vec2_labels_encoded)
assert np.array_equal(egemaps_labels_encoded, wavlm_labels_encoded)

# ✅ Concatenate features
combined_features = np.concatenate(
    [egemaps_features, wav2vec2_features, wavlm_features], axis=1
)
combined_labels = egemaps_labels_encoded

print("\n✅ Combined feature shape:", combined_features.shape)
print("✅ Combined label shape:", combined_labels.shape)
print("✅ Classes:", le.classes_)


Unique labels per source:
eGeMAPS: ['Control' 'Dementia']
Wav2Vec2: ['0' '1']
WavLM: ['Control' 'Dementia']


ValueError: y contains previously unseen labels: '0'

In [4]:
# ============================================
# Cell 3: Concatenate All Features
# ============================================

# Ensure all labels match
assert np.array_equal(egemaps_labels, wav2vec2_labels)
assert np.array_equal(egemaps_labels, wavlm_labels)

# Concatenate features along axis 1 (feature dimension)
combined_features = np.concatenate([egemaps_features, wav2vec2_features, wavlm_features], axis=1)
combined_labels = egemaps_labels  # all are same

print("✅ Combined feature shape:", combined_features.shape)
print("✅ Combined labels shape:", combined_labels.shape)


c:\Users\admin\Desktop\major major 6\major 6 final new\agraja_env\lib\site-packages\numpy\core\numeric.py:2468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


AssertionError: 

In [3]:
# ============================================
# Cell 4: CNN Definition
# ============================================

class SimpleCNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleCNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
# ============================================
# Cell 5: 10-Fold Cross Validation (Combined CNN)
# ============================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️ Using device:", device)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(combined_labels)
num_classes = len(np.unique(y_encoded))
input_dim = combined_features.shape[1]

# Scale features
X_scaled = StandardScaler().fit_transform(combined_features)

# 10-fold setup
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
all_true, all_pred = [], []

# Define model, loss, optimizer
model = SimpleCNN(input_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for fold, (train_idx, test_idx) in enumerate(kf.split(X_scaled, y_encoded), 1):
    print(f"\n🔹 Fold {fold}/10")
    
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Convert to tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

    # Dataloader
    train_ds = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

    # Training
    model.train()
    for epoch in range(10):
        epoch_loss = 0
        for xb, yb in train_loader:
            optimizer.zero_grad()
            outputs = model(xb)
            loss = criterion(outputs, yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/10 - Loss: {epoch_loss/len(train_loader):.4f}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        preds = torch.argmax(model(X_test_tensor), dim=1)
        all_true.extend(y_test_tensor.cpu().numpy())
        all_pred.extend(preds.cpu().numpy())

# Save final model
save_dir = os.path.join("saved_models", "combined_features")
os.makedirs(save_dir, exist_ok=True)
torch.save(model.state_dict(), os.path.join(save_dir, "CNN_combined_final.pth"))

print("\n✅ Model training complete and saved.")


In [ ]:
# ============================================
# Cell 6: Evaluation Results
# ============================================

cm = confusion_matrix(all_true, all_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Combined Features - CNN Confusion Matrix")
plt.show()

print("\n📈 Classification Report:")
print(classification_report(all_true, all_pred, digits=3))
